In [1]:
from torchvision import transforms
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from PIL import Image

In [2]:
def squeeze(img):
    img = np.array(img).astype(float) / 255.0
    img = Image.fromarray(img, mode='L')
    return img

In [12]:
transform = transforms.Compose([
    transforms.Grayscale(),
    squeeze,
    transforms.ToTensor(),
    transforms.Normalize([0.5], [1])
])

In [14]:
BATCH_SIZE = 128

train = torchvision.datasets.ImageFolder('../data/Training/', transform=transform)
test = torchvision.datasets.ImageFolder('../data/PublicTest/', transform=transform)

trainData = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
testData = torch.utils.data.DataLoader(test, batch_size=3000, shuffle=False)

In [19]:
# define model
class Net(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=1, stride=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # self.conv4 = nn.Conv2d(32, 64, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(1152 * 2, 512)
        self.fc2 = nn.Linear(512, 7)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        
        x = x.view(x.shape[0], -1)
        x = F.dropout(x, 0.25)
        x = self.fc1(x)
        x = F.relu(x)
        
        x = F.dropout(x, 0.25)
        x = self.fc2(x)
        return x
    
model = Net()

In [20]:
# optimizer and loss
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
optimizer = optim.SGD(model.parameters(), lr=1e-2)

In [17]:
def one_hot(label, n_category=None):
    batch_size = label.shape[0]
    n_category = n_category or (label.max().item() + 1)

    y_onehot = torch.zeros((batch_size, n_category), dtype=torch.long)
    y_onehot.scatter_(1, label.view(-1, 1), 1)
    return y_onehot

In [21]:
# train
import tqdm

running_loss = 0.0
for epoch in range(3):
    print('EPOCH {}'.format(epoch + 1))
    
    for i, data in tqdm.tqdm(enumerate(trainData)):
        inputs, label = data
        # label = one_hot(label)
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, label)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:
            # loss on testing set
            model.eval()
            test_input, test_label = next(iter(testData))
            pred_test = model(test_input)
            test_loss = criterion(pred_test, test_label)
            
            print('Train loss: %.3f, Test loss: %.3f' %
                  (running_loss / 100, test_loss))
            
            running_loss = 0.0
            model.train()
            
            

EPOCH 1


99it [02:19,  1.34s/it]

Train loss: 1.853, Test loss: 1.846


199it [05:32,  1.32s/it]

RuntimeError: $ Torch: not enough memory: you tried to allocate 5GB. Buy new RAM! at ..\aten\src\TH\THGeneral.cpp:201